In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from pathlib import Path
import pandas as pd


MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
DATA_DIR = Path("/kaggle/input/ts-pmo/vihallu-train_part_1.csv")
OUTPUT_DIR = Path("vihallu-train-corrected.csv")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
)

In [ ]:
def correct_prompt(context, prompt, response, model, tokenizer):
    sys_prompt = (
        "Bạn là một bộ sửa chính tả tiếng Việt. "
        "Chỉ sửa lỗi CHÍNH TẢ/đánh máy và thêm dấu cho các từ ngữ trong PROMPT, có thể dùng CONTEXT/RESPONSE để đoán chữ đúng. "
        "KHÔNG thêm ý mới, KHÔNG đổi nội dung nghĩa, KHÔNG liệt kê hay giải thích. "
        "Nếu không cần sửa thì trả về nguyên văn PROMPT. "
        "Chỉ trả về MỘT DÒNG là câu PROMPT đã được sửa, không thêm dấu ngoặc hay tiền tố nào."
    )

    user_prompt = (
        f"CONTEXT:\n{context}\n\n"
        f"PROMPT:\n{prompt}\n\n"
        f"RESPONSE:\n{response}\n"
        "Yêu cầu: Trả về đúng MỘT câu PROMPT đã được sửa lỗi chính tả."
    )

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True,
    ).to(model.device)

    attention_mask = (
        (inputs != tokenizer.pad_token_id).long()
        if tokenizer.pad_token_id is not None
        else None
    )

    with torch.no_grad():
        generation_output = model.generate(
            inputs,
            attention_mask=attention_mask,
            max_new_tokens=256,
            do_sample=False,
            temperature=0.0,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1.0,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    new_tokens = generation_output[0, inputs.shape[-1]:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    # print("Corrected Prompt:", text)

    return text

In [ ]:
data = pd.read_csv(DATA_DIR)

def get_item_by_id(data, id) -> dict:
    item = data[data['id'] == id].iloc[0]

    return {
        'context': item['context'],
        'prompt': item['prompt'],
        'response': item['response']
    }

In [ ]:
from tqdm import tqdm

results = []

for idx, row in tqdm(data.iterrows(), total=len(data)):
    corrected = correct_prompt(
        row["context"], row["prompt"], row["response"], model, tokenizer
    )
    results.append(
        {
            "id": row["id"],
            "context": row["context"],
            "prompt": row["prompt"],
            "corrected_prompt": corrected,
            "response": row["response"],
        }
    )

In [ ]:
df = pd.DataFrame(results)
df.to_csv(OUTPUT_DIR, index=False)